# Recurrent Neural Network (RNN) Examples:

**Source (all credits to):** *Easy TensorFlow*

<img src="../resources/r_nn_fixed.png" style="width:80%">

2 examples:

1. RNN with **fixed** sequence length.
2. RNN with **variable** sequence length.

# 1. Import Libraries

In [63]:
import tensorflow as tf
import numpy as np
import keras
import keras.backend as K

# use TensorFlow v.1
import tensorflow.compat.v1 as tf 
tf.disable_v2_behavior()

# 2. Prepare Data

## 2.1. Generate Data

In [64]:
x_train = np.random.randint(0, 10, size=(100, 4, 1))
y_train = np.sum(x_train, axis=1)

x_test = np.random.randint(0, 10, size=(5, 4, 1))
y_test = np.sum(x_test, axis=1)

print("Size of:")
print("- Training-set size:\t\t{}".format(len(y_train)))
print("- Test-set size:\t{}".format(len(y_test)))

Size of:
- Training-set size:		100
- Test-set size:	5


## 2.2. Look at Data

In [65]:
print('Train TOP 5:')
for idx in range(0, 5):
    print('X_train: {}; y_train: {}'.format(x_train[idx].T, y_train[idx]))
    
print('---')
print('Test TOP 5:')
for idx in range(0, 5):
    print('X_test: {}; y_test: {}'.format(x_test[idx].T, y_test[idx]))

Train TOP 5:
X_train: [[2 2 0 9]]; y_train: [13]
X_train: [[1 0 0 6]]; y_train: [7]
X_train: [[4 5 8 7]]; y_train: [24]
X_train: [[6 7 4 1]]; y_train: [18]
X_train: [[7 2 8 1]]; y_train: [18]
---
Test TOP 5:
X_test: [[0 9 6 2]]; y_test: [17]
X_test: [[7 2 7 1]]; y_test: [17]
X_test: [[3 7 5 8]]; y_test: [23]
X_test: [[1 3 0 7]]; y_test: [11]
X_test: [[4 5 1 2]]; y_test: [12]


# 3. Example 1 - RNN with Fixed Sequence Length

In [66]:
# data dimensions
input_dim = 1
seq_max_len = len(x_train[0]) # sequence maximum length
out_dim = len(y_train[0])

# clear context
K.clear_session()

# hyper-parameters
EPOCHS = 10000
BATCH_SIZE = 10
DISPLAY_FREQ = 1000      # frequency of displaying the training results
LEARNING_RATE = 0.001
NUM_HIDDEN_UNITS = 10    # number of hidden units

# model input data
x = tf.placeholder(tf.float32, shape=[None, seq_max_len, input_dim], name='X')
y = tf.placeholder(tf.float32, shape=[None, input_dim], name='Y')
 
# model parameters: W (weights) and b (bias)
W = weight_variable(shape=[NUM_HIDDEN_UNITS, out_dim])
b = bias_variable(shape=[out_dim])

# calculate logits (forward output)
pred_out = RNN(x, W, b, NUM_HIDDEN_UNITS)

######## prepare model ########

# loss function = mean-squared error
cost = tf.reduce_mean(tf.square(pred_out - y))

# optimizer = Adam
optimizer = tf.train.AdamOptimizer(learning_rate=LEARNING_RATE, name='Adam-op').minimize(cost)
 
# initialize tf variables
init = tf.global_variables_initializer()
 
######## train model ########

# create a tf session
sess = tf.InteractiveSession()

# initialize session variables
sess.run(init)

for i in range(EPOCHS):
    x_batch, y_batch = next_batch(x_train, y_train, BATCH_SIZE)
    _, mse = sess.run([optimizer, cost], feed_dict={x: x_batch, y: y_batch})
    
    if i % DISPLAY_FREQ == 0:
        print('Step {}, MSE={}'.format(i, mse))
        
# validation
y_pred = sess.run(pred_out, feed_dict={x: x_test})
    
######## test model ########

y_pred = sess.run(pred_out, feed_dict={x: x_test})

print('---')
print('Test:')

for i, x in enumerate(y_test):
    print("When the ground truth output is {}, the model thinks it is {}"
          .format(y_test[i], y_pred[i]))

sess.close()

/root/anaconda3/lib/python3.8/site-packages/tensorflow/python/client/session.py:1761: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


Step 0, MSE=253.41464233398438
Step 1000, MSE=170.3509979248047
Step 2000, MSE=31.76686668395996
Step 3000, MSE=23.367515563964844
Step 4000, MSE=1.276302456855774
Step 5000, MSE=0.9381116032600403
Step 6000, MSE=0.09752456843852997
Step 7000, MSE=0.5929590463638306
Step 8000, MSE=0.10749709606170654
Step 9000, MSE=0.22929659485816956
---
Test:
When the ground truth output is [17], the model thinks it is [17.367632]
When the ground truth output is [17], the model thinks it is [17.386267]
When the ground truth output is [23], the model thinks it is [22.94692]
When the ground truth output is [11], the model thinks it is [11.077478]
When the ground truth output is [12], the model thinks it is [12.041155]


# 4. Example 2 - RNN with Variable Sequence Length

In [67]:
# data dimensions
input_dim = 1
seq_max_len = len(x_train[0]) # sequence maximum length
out_dim = len(y_train[0])
seq_len_train = np.random.randint(1, seq_max_len+1, len(y_train))
seq_len_test = np.random.randint(1, seq_max_len+1, len(y_test))

# clear context
K.clear_session()

# hyper-parameters
EPOCHS = 10000
BATCH_SIZE = 10
DISPLAY_FREQ = 1000      # frequency of displaying the training results
LEARNING_RATE = 0.001
NUM_HIDDEN_UNITS = 10    # number of hidden units

# model input data
x = tf.placeholder(tf.float32, shape=[None, seq_max_len, input_dim], name='X')
seqLen = tf.placeholder(tf.int32, [None])
y = tf.placeholder(tf.float32, shape=[None, input_dim], name='Y')
 
# model parameters: W (weights) and b (bias)
W = weight_variable(shape=[NUM_HIDDEN_UNITS, out_dim])
b = bias_variable(shape=[out_dim])

# calculate logits (forward output)
pred_out = RNN_2(x, W, b, NUM_HIDDEN_UNITS, seq_max_len, seqLen)

######## prepare model ########

# loss function = mean-squared error
cost = tf.reduce_mean(tf.square(pred_out - y))

# optimizer = Adam
optimizer = tf.train.AdamOptimizer(learning_rate=LEARNING_RATE, name='Adam-op').minimize(cost)
 
# initialize tf variables
init = tf.global_variables_initializer()
 
######## train model ########

# with: create a tf session, and initialize session variables
with tf.Session() as sess:
    sess.run(init)
    
    print('----------Training---------')
    for i in range(EPOCHS):
        x_batch, y_batch, seq_len_batch = next_batch_2(x_train, y_train, seq_len_train, BATCH_SIZE)
        _, mse = sess.run([optimizer, cost], feed_dict={x: x_batch, y: y_batch, seqLen: seq_len_batch})
        
        if i % DISPLAY_FREQ == 0:
            print('Step {0:<6}, MSE={1:.4f}'.format(i, mse))
    
    ######## test model ########
    y_pred = sess.run(pred_out, feed_dict={x: x_test, seqLen: seq_len_test})
    
    print()
    print('--------Test Results-------')
    for i, x in enumerate(y_test):
        print("When the ground truth output is {}, the model thinks it is {}"
              .format(y_test[i], y_pred[i]))

----------Training---------
Step 0     , MSE=377.3354
Step 1000  , MSE=108.3779
Step 2000  , MSE=22.5980
Step 3000  , MSE=33.2016
Step 4000  , MSE=10.7104
Step 5000  , MSE=9.9249
Step 6000  , MSE=22.7768
Step 7000  , MSE=12.8159
Step 8000  , MSE=12.4850
Step 9000  , MSE=24.1962

--------Test Results-------
When the ground truth output is [17], the model thinks it is [19.231289]
When the ground truth output is [17], the model thinks it is [18.118595]
When the ground truth output is [23], the model thinks it is [19.52219]
When the ground truth output is [11], the model thinks it is [13.784103]
When the ground truth output is [12], the model thinks it is [11.877954]


# 5. Functions Used

In [68]:
def next_batch(x, y, batch_size):
    N = x.shape[0]
    batch_indices = np.random.permutation(N)[:batch_size]
    x_batch = x[batch_indices]
    y_batch = y[batch_indices]
    return x_batch, y_batch

# weight and bais wrappers
def weight_variable(shape):
    """
    Create a weight variable with appropriate initialization
    :param name: weight name
    :param shape: weight shape
    :return: initialized weight variable
    """
    initer = tf.truncated_normal_initializer(stddev=0.01)
    return tf.get_variable('W',
                           dtype=tf.float32,
                           shape=shape,
                           initializer=initer)

def bias_variable(shape):
    """
    Create a bias variable with appropriate initialization
    :param name: bias variable name
    :param shape: bias variable shape
    :return: initialized bias variable
    """
    initial = tf.constant(0., shape=shape, dtype=tf.float32)
    return tf.get_variable('b',
                           dtype=tf.float32,
                           initializer=initial)
def RNN(x, weights, biases, num_hidden):
    """
    :param x: inputs of size [batch_size, max_time, input_dim]
    :param weights: matrix of fully-connected output layer weights
    :param biases: vector of fully-connected output layer biases
    :param num_hidden: number of hidden units
    """
    cell = tf.nn.rnn_cell.BasicRNNCell(num_hidden)
    outputs, states = tf.nn.dynamic_rnn(cell, x, dtype=tf.float32)
    out = tf.matmul(outputs[:, -1, :], weights) + biases
    return out

def RNN_2(x, weights, biases, n_hidden, seq_max_len, seq_len):
    """
    :param x: inputs of shape [batch_size, max_time, input_dim]
    :param weights: matrix of fully-connected output layer weights
    :param biases: vector of fully-connected output layer biases
    :param n_hidden: number of hidden units
    :param seq_max_len: sequence maximum length
    :param seq_len: length of each sequence of shape [batch_size,]
    """
    cell = tf.nn.rnn_cell.BasicRNNCell(n_hidden)
    outputs, states = tf.nn.dynamic_rnn(cell, x, sequence_length=seq_len, dtype=tf.float32)

    # Hack to build the indexing and retrieve the right output.
    batch_size = tf.shape(outputs)[0]
    # Start indices for each sample
    index = tf.range(0, batch_size) * seq_max_len + (seq_len - 1)
    # Indexing
    outputs = tf.gather(tf.reshape(outputs, [-1, n_hidden]), index)
    out = tf.matmul(outputs, weights) + biases
    return out

def next_batch_2(x, y, seq_len, batch_size):
    N = x.shape[0]
    batch_indeces = np.random.permutation(N)[:batch_size]
    x_batch = x[batch_indeces]
    y_batch = y[batch_indeces]
    seq_len_batch = seq_len[batch_indeces]
    return x_batch, y_batch, seq_len_batch